In [126]:
import tkinter as tk
from tkinter import messagebox
from functools import partial
import math

# Constants and Global Variables
EMPTY = 0
BLACK = 1
WHITE = 2
board = None
buttons = None
current_player = None
depth = 1  # Default depth for the AI search

# Difficulty levels mapping
difficulty_levels = {
    "easy": 1,
    "medium": 3,
    "hard": 5
}

# Board Initialization and GUI Functions
def create_board(root, make_move_func):
    buttons = [[None] * 8 for _ in range(8)]
    for r in range(8):
        for c in range(8):
            buttons[r][c] = tk.Button(root, text="", width=2, height=1,
                                       command=partial(make_move_func, r, c))
            buttons[r][c].grid(row=r, column=c)
    return buttons

def update_board(board, buttons):
    for r in range(8):
        for c in range(8):
            color = "green" if board[r][c] == EMPTY else ("black" if board[r][c] == BLACK else "white")
            state = "normal" if board[r][c] == EMPTY else "disabled"
            buttons[r][c].configure(bg=color, state=state)

# Move Validation and Execution Functions
def is_valid_move(board, row, col, current_player):
    if board[row][col] != EMPTY:
        return False
    directions = [(0, 1), (1, 0), (0, -1), (-1, 0), (1, 1), (1, -1), (-1, 1), (-1, -1)]
    opponent = WHITE if current_player == BLACK else BLACK
    for dr, dc in directions:
        r, c = row + dr, col + dc
        if not (0 <= r < 8 and 0 <= c < 8) or board[r][c] != opponent:
            continue
        while 0 <= r < 8 and 0 <= c < 8 and board[r][c] == opponent:
            r, c = r + dr, c + dc
        if 0 <= r < 8 and 0 <= c < 8 and board[r][c] == current_player:
            return True
    return False

def ai_make_move():
    global current_player, depth
    if not any(is_valid_move(board, r, c, WHITE) for r in range(8) for c in range(8)):
        current_player = BLACK
        return
    move = ai_move(board, WHITE, depth)  # Get the AI move
    if move is None:
        current_player = BLACK  # Skip AI's turn if no move is possible
        return
    row, col = move
    board[row][col] = WHITE
    flip_pieces(row, col, WHITE)  # Flip opponent's pieces
    update_board(board, buttons)
    if is_game_over(board):
        show_winner(board)
    else:
        current_player = BLACK  # Switch to player's turn

def flip_pieces(row, col, player):
    directions = [(0, 1), (1, 0), (0, -1), (-1, 0), (1, 1), (1, -1), (-1, 1), (-1, -1)]
    opponent = WHITE if player == BLACK else BLACK
    for dr, dc in directions:
        r, c = row + dr, col + dc
        flip_list = []
        while 0 <= r < 8 and 0 <= c < 8 and board[r][c] == opponent:
            flip_list.append((r, c))
            r, c = r + dr, c + dc
        if 0 <= r < 8 and 0 <= c < 8 and board[r][c] == player:
            for flip_row, flip_col in flip_list:
                board[flip_row][flip_col] = player

# Game Over and Winner Determination
def is_game_over(board):
    return not any(is_valid_move(board, r, c, player) for r in range(8) for c in range(8) for player in [BLACK, WHITE])

def show_winner(board):
    black_count = sum(row.count(BLACK) for row in board)
    white_count = sum(row.count(WHITE) for row in board)
    winner = "BLACK" if black_count > white_count else ("WHITE" if black_count < white_count else "It's a tie!")
    messagebox.showinfo("Game Over", f"{winner} wins!")

# AI Algorithm (Alpha-Beta Pruning)
def alpha_beta(board, depth, alpha, beta, maximizing_player, player):
    if depth == 0 or is_game_over(board):
        return evaluate(board, player)
    
    for r in range(8):
        for c in range(8):
            if is_valid_move(board, r, c, player):
                new_board = [row[:] for row in board]
                new_board[r][c] = player
                eval = alpha_beta(new_board, depth - 1, alpha, beta, not maximizing_player, player)
                if maximizing_player:
                    alpha = max(alpha, eval)
                    if beta <= alpha:
                        return eval
                else:
                    beta = min(beta, eval)
                    if beta <= alpha:
                        return eval
    return alpha if maximizing_player else beta

def evaluate(board, player):
    black_count = sum(row.count(BLACK) for row in board)
    white_count = sum(row.count(WHITE) for row in board)
    return black_count - white_count if player == BLACK else white_count - black_count

def ai_move(board, player, depth):
    best_eval = -math.inf
    best_move = None
    for r in range(8):
        for c in range(8):
            if is_valid_move(board, r, c, player):
                new_board = [row[:] for row in board]
                new_board[r][c] = player
                eval = alpha_beta(new_board, depth, -math.inf, math.inf, False, player)
                if eval > best_eval:
                    best_eval = eval
                    best_move = (r, c)
    return best_move if best_move is not None else None  # Return None if no valid move is found

# GUI Initialization
def select_difficulty(difficulty):
    global depth
    depth = difficulty_levels[difficulty]

def start_game():
    global board, buttons, current_player
    root.destroy()  # Close the difficulty selection window
    root_game = tk.Tk()
    root_game.title("Othello")

    # Board GUI
    board_frame = tk.Frame(root_game)
    board_frame.pack()
    board = [[EMPTY] * 8 for _ in range(8)]
    board[3][3] = board[4][4] = WHITE
    board[3][4] = board[4][3] = BLACK
    buttons = create_board(board_frame, make_move)
    update_board(board, buttons)

    root_game.mainloop()

def make_move(row, col):
    global current_player
    if current_player != BLACK or not is_valid_move(board, row, col, BLACK):
        return
    board[row][col] = BLACK
    flip_pieces(row, col, BLACK)  # Flip opponent's pieces
    update_board(board, buttons)
    if is_game_over(board):
        show_winner(board)
    else:
        current_player = WHITE  # Switch to AI's turn
        ai_make_move()  # AI makes a move

if __name__ == "__main__":
    current_player = BLACK

    # Difficulty Selection GUI
    root = tk.Tk()
    root.title("Select Difficulty")
    difficulty_frame = tk.Frame(root)
    difficulty_frame.pack()
    for difficulty in difficulty_levels:
        tk.Button(difficulty_frame, text=difficulty, command=partial(select_difficulty, difficulty)).pack()
    start_button = tk.Button(root, text="Start Game", command=start_game)
    start_button.pack()

    root.mainloop()
